# Stage 03: Python Fundamentals for Quantitative Finance
**Project:** Turtle Trading Strategy Research  
**Author:** Panwei Hu  
**Date:** 2025-08-17

## Objectives
- Demonstrate core Python skills for quantitative finance
- Implement financial calculations and time series operations
- Build foundation functions for Turtle Trading strategy
- Practice data manipulation and analysis patterns
- Create reusable components for the research pipeline

## Focus Areas
- **Time Series Analysis**: Price data, returns, volatility calculations
- **Financial Metrics**: Sharpe ratio, drawdown, performance statistics
- **Technical Indicators**: Moving averages, Donchian channels, ATR
- **Risk Management**: Position sizing, portfolio allocation
- **Data Structures**: Efficient handling of multi-asset datasets


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up project environment
PROJECT_ROOT = Path('..').resolve()
import sys
sys.path.append(str(PROJECT_ROOT / 'src'))

print("🐢 Python Fundamentals for Turtle Trading")
print("="*50)

# Configure plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("✅ Libraries imported and environment configured")


🐢 Python Fundamentals for Turtle Trading
✅ Libraries imported and environment configured


In [2]:
# Core Financial Calculations
class FinancialMetrics:
    """Core financial calculations for quantitative analysis"""
    
    @staticmethod
    def returns(prices: pd.Series, method: str = 'simple') -> pd.Series:
        """Calculate returns from price series"""
        if method == 'simple':
            return prices.pct_change()
        elif method == 'log':
            return np.log(prices / prices.shift(1))
        else:
            raise ValueError("Method must be 'simple' or 'log'")
    
    @staticmethod
    def volatility(returns: pd.Series, annualize: bool = True, periods: int = 252) -> float:
        """Calculate annualized volatility"""
        vol = returns.std()
        return vol * np.sqrt(periods) if annualize else vol
    
    @staticmethod
    def sharpe_ratio(returns: pd.Series, risk_free: float = 0.0, periods: int = 252) -> float:
        """Calculate Sharpe ratio"""
        excess_returns = returns - risk_free / periods
        return np.sqrt(periods) * excess_returns.mean() / returns.std()
    
    @staticmethod
    def max_drawdown(prices: pd.Series) -> Dict[str, float]:
        """Calculate maximum drawdown and related metrics"""
        cumulative = (1 + FinancialMetrics.returns(prices)).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        
        max_dd = drawdown.min()
        max_dd_date = drawdown.idxmin()
        
        return {
            'max_drawdown': max_dd,
            'max_drawdown_date': max_dd_date,
            'current_drawdown': drawdown.iloc[-1]
        }
    
    @staticmethod
    def calmar_ratio(returns: pd.Series, periods: int = 252) -> float:
        """Calculate Calmar ratio (annual return / max drawdown)"""
        annual_return = (1 + returns).prod() ** (periods / len(returns)) - 1
        prices = (1 + returns).cumprod()
        max_dd = abs(FinancialMetrics.max_drawdown(prices)['max_drawdown'])
        
        return annual_return / max_dd if max_dd != 0 else np.inf

# Test with sample data
print("📊 Testing Financial Metrics:")

# Generate sample price data
np.random.seed(42)
dates = pd.date_range('2020-01-01', '2024-12-31', freq='D')
returns = np.random.normal(0.0005, 0.02, len(dates))  # Daily returns with drift
prices = pd.Series(100 * (1 + returns).cumprod(), index=dates, name='price')

# Calculate metrics
sample_returns = FinancialMetrics.returns(prices)
vol = FinancialMetrics.volatility(sample_returns)
sharpe = FinancialMetrics.sharpe_ratio(sample_returns)
dd_info = FinancialMetrics.max_drawdown(prices)
calmar = FinancialMetrics.calmar_ratio(sample_returns)

print(f"Sample period: {prices.index[0].date()} to {prices.index[-1].date()}")
print(f"Total return: {(prices.iloc[-1] / prices.iloc[0] - 1) * 100:.2f}%")
print(f"Annualized volatility: {vol * 100:.2f}%")
print(f"Sharpe ratio: {sharpe:.3f}")
print(f"Maximum drawdown: {dd_info['max_drawdown'] * 100:.2f}%")
print(f"Calmar ratio: {calmar:.3f}")

print("\n✅ Financial metrics module working")


📊 Testing Financial Metrics:
Sample period: 2020-01-01 to 2024-12-31
Total return: 724.97%
Annualized volatility: 31.43%
Sharpe ratio: 1.084
Maximum drawdown: -42.33%
Calmar ratio: 0.798

✅ Financial metrics module working


In [ ]:
# Technical Indicators for Turtle Trading
class TechnicalIndicators:
    """Technical indicators specifically for Turtle Trading strategy"""
    
    @staticmethod
    def donchian_channels(prices: pd.Series, period: int = 20) -> Dict[str, pd.Series]:
        """Calculate Donchian channel (key component of Turtle system)"""
        high_channel = prices.rolling(window=period).max()
        low_channel = prices.rolling(window=period).min()
        mid_channel = (high_channel + low_channel) / 2
        
        return {
            'upper': high_channel,
            'lower': low_channel,
            'middle': mid_channel
        }
    
    @staticmethod
    def atr(high: pd.Series, low: pd.Series, close: pd.Series, period: int = 20) -> pd.Series:
        """Calculate Average True Range (ATR) for position sizing"""
        # True Range calculation
        tr1 = high - low
        tr2 = abs(high - close.shift(1))
        tr3 = abs(low - close.shift(1))
        
        true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = true_range.rolling(window=period).mean()
        
        return atr
    
    @staticmethod
    def moving_average(prices: pd.Series, period: int, method: str = 'simple') -> pd.Series:
        """Calculate moving averages"""
        if method == 'simple':
            return prices.rolling(window=period).mean()
        elif method == 'exponential':
            return prices.ewm(span=period).mean()
        else:
            raise ValueError("Method must be 'simple' or 'exponential'")
    
    @staticmethod
    def breakout_signals(prices: pd.Series, entry_period: int = 20, exit_period: int = 10) -> Dict[str, pd.Series]:
        """Generate Turtle Trading breakout signals"""
        # Entry signals
        entry_channels = TechnicalIndicators.donchian_channels(prices, entry_period)
        long_entry = prices > entry_channels['upper'].shift(1)
        short_entry = prices < entry_channels['lower'].shift(1)
        
        # Exit signals  
        exit_channels = TechnicalIndicators.donchian_channels(prices, exit_period)
        long_exit = prices < exit_channels['lower'].shift(1)
        short_exit = prices > exit_channels['upper'].shift(1)
        
        return {
            'long_entry': long_entry,
            'short_entry': short_entry,
            'long_exit': long_exit,
            'short_exit': short_exit,
            'entry_upper': entry_channels['upper'],
            'entry_lower': entry_channels['lower'],
            'exit_upper': exit_channels['upper'],
            'exit_lower': exit_channels['lower']
        }

# Test technical indicators - FIXED VERSION
print("🔧 Testing Technical Indicators:")

# Create more realistic price data with OHLC - FIXED
np.random.seed(42)
n_days = 500
base_price = 100
dates = pd.date_range('2023-01-01', periods=n_days, freq='D')

# Generate price changes and create pandas Series first - THIS IS THE FIX!
price_changes = np.random.normal(0, 0.02, n_days)
closes = pd.Series(base_price * (1 + price_changes).cumprod(), index=dates)

# Generate OHLC data using pandas Series methods
highs = closes * (1 + np.abs(np.random.normal(0, 0.01, n_days)))
lows = closes * (1 - np.abs(np.random.normal(0, 0.01, n_days)))
opens = closes.shift(1).fillna(base_price)  # Now this works because closes is a pandas Series

ohlc_data = pd.DataFrame({
    'open': opens,
    'high': highs, 
    'low': lows,
    'close': closes
})

# Test Donchian channels
donchian = TechnicalIndicators.donchian_channels(ohlc_data['close'], 20)
print(f"Donchian channels calculated for {len(donchian['upper'])} periods")

# Test ATR
atr_values = TechnicalIndicators.atr(ohlc_data['high'], ohlc_data['low'], ohlc_data['close'])
avg_atr = atr_values.mean()
print(f"Average ATR: {avg_atr:.4f} ({avg_atr/ohlc_data['close'].mean()*100:.2f}% of price)")

# Test breakout signals
signals = TechnicalIndicators.breakout_signals(ohlc_data['close'])
long_signals = signals['long_entry'].sum()
short_signals = signals['short_entry'].sum()
print(f"Generated {long_signals} long entry signals and {short_signals} short entry signals")

# Test moving averages
ma_20 = TechnicalIndicators.moving_average(ohlc_data['close'], 20)
ma_50 = TechnicalIndicators.moving_average(ohlc_data['close'], 50)
print(f"Moving averages: MA20={ma_20.iloc[-1]:.2f}, MA50={ma_50.iloc[-1]:.2f}")

print("\n✅ Technical indicators module working")


🔧 Testing Technical Indicators:
Donchian channels calculated for 500 periods
Average ATR: 2.3617 (2.52% of price)
Generated 52 long entry signals and 60 short entry signals
Moving averages: MA20=100.52, MA50=100.48

✅ Technical indicators module working


In [10]:
# Risk Management and Position Sizing
class RiskManagement:
    """Risk management utilities for Turtle Trading"""
    
    @staticmethod
    def position_size_atr(account_value: float, risk_per_trade: float, atr: float, price: float) -> int:
        """Calculate position size based on ATR (Turtle method)"""
        # Dollar risk per trade
        dollar_risk = account_value * risk_per_trade
        
        # Position size = Dollar risk / (ATR * multiplier)
        # Using 2 * ATR as stop distance (common Turtle approach)
        stop_distance = 2 * atr
        position_size = int(dollar_risk / stop_distance)
        
        return max(position_size, 0)
    
    @staticmethod
    def portfolio_allocation(symbols: List[str], equal_weight: bool = True, 
                           custom_weights: Optional[Dict[str, float]] = None) -> Dict[str, float]:
        """Calculate portfolio allocation weights"""
        if equal_weight:
            weight = 1.0 / len(symbols)
            return {symbol: weight for symbol in symbols}
        elif custom_weights:
            total_weight = sum(custom_weights.values())
            return {symbol: weight/total_weight for symbol, weight in custom_weights.items()}
        else:
            raise ValueError("Must specify either equal_weight=True or custom_weights")
    
    @staticmethod
    def max_position_limits(account_value: float, max_risk_per_position: float = 0.02,
                          max_sector_risk: float = 0.20) -> Dict[str, float]:
        """Calculate maximum position and sector limits"""
        return {
            'max_position_value': account_value * max_risk_per_position,
            'max_sector_value': account_value * max_sector_risk,
            'max_portfolio_risk': account_value * 0.02  # 2% portfolio stop
        }
    
    @staticmethod
    def correlation_analysis(returns_df: pd.DataFrame) -> Dict[str, Union[pd.DataFrame, float]]:
        """Analyze correlations between assets for diversification"""
        corr_matrix = returns_df.corr()
        
        # Average correlation (excluding diagonal)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        avg_correlation = corr_matrix.where(mask).stack().mean()
        
        # Find highly correlated pairs (>0.7)
        high_corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > 0.7:
                    high_corr_pairs.append({
                        'asset1': corr_matrix.columns[i],
                        'asset2': corr_matrix.columns[j], 
                        'correlation': corr_val
                    })
        
        return {
            'correlation_matrix': corr_matrix,
            'average_correlation': avg_correlation,
            'high_correlation_pairs': high_corr_pairs
        }

# Test risk management functions
print("⚖️  Testing Risk Management:")

# Portfolio setup
account_value = 1000000  # $1M account
symbols = ['SPY', 'QQQ', 'GLD', 'TLT', 'EEM']
risk_per_trade = 0.01  # 1% risk per trade

# Test position sizing
test_atr = 2.5
test_price = 150.0
position_size = RiskManagement.position_size_atr(account_value, risk_per_trade, test_atr, test_price)
print(f"Position size for ATR={test_atr}, Price=${test_price}: {position_size} shares")
print(f"Position value: ${position_size * test_price:,.0f} ({position_size * test_price / account_value * 100:.1f}% of account)")

# Test portfolio allocation
equal_weights = RiskManagement.portfolio_allocation(symbols, equal_weight=True)
print(f"\nEqual weight allocation: {equal_weights}")

custom_weights = {'SPY': 0.3, 'QQQ': 0.2, 'GLD': 0.2, 'TLT': 0.2, 'EEM': 0.1}
custom_allocation = RiskManagement.portfolio_allocation(symbols, equal_weight=False, custom_weights=custom_weights)
print(f"Custom allocation: {custom_allocation}")

# Test position limits
limits = RiskManagement.max_position_limits(account_value)
print(f"\nPosition limits:")
for limit_type, value in limits.items():
    print(f"  {limit_type}: ${value:,.0f}")

# Test correlation analysis with synthetic data
np.random.seed(42)
n_days = 252
returns_data = {}

# Create correlated returns for testing
base_returns = np.random.normal(0, 0.02, n_days)
for i, symbol in enumerate(symbols):
    # Add some correlation structure
    correlation_factor = 0.3 + i * 0.1
    symbol_returns = correlation_factor * base_returns + (1 - correlation_factor) * np.random.normal(0, 0.02, n_days)
    returns_data[symbol] = symbol_returns

returns_df = pd.DataFrame(returns_data)
corr_analysis = RiskManagement.correlation_analysis(returns_df)

print(f"\nCorrelation Analysis:")
print(f"Average correlation: {corr_analysis['average_correlation']:.3f}")
print(f"High correlation pairs: {len(corr_analysis['high_correlation_pairs'])}")

for pair in corr_analysis['high_correlation_pairs']:
    print(f"  {pair['asset1']} - {pair['asset2']}: {pair['correlation']:.3f}")

print("\n✅ Risk management module working")


⚖️  Testing Risk Management:
Position size for ATR=2.5, Price=$150.0: 2000 shares
Position value: $300,000 (30.0% of account)

Equal weight allocation: {'SPY': 0.2, 'QQQ': 0.2, 'GLD': 0.2, 'TLT': 0.2, 'EEM': 0.2}
Custom allocation: {'SPY': 0.30000000000000004, 'QQQ': 0.20000000000000004, 'GLD': 0.20000000000000004, 'TLT': 0.20000000000000004, 'EEM': 0.10000000000000002}

Position limits:
  max_position_value: $20,000
  max_sector_value: $200,000
  max_portfolio_risk: $20,000

Correlation Analysis:
Average correlation: 0.473
High correlation pairs: 1
  TLT - EEM: 0.768

✅ Risk management module working


In [ ]:
# Data Processing and Portfolio Management
class PortfolioManager:
    """Portfolio management utilities for multi-asset strategies"""
    
    def __init__(self, symbols: List[str], initial_capital: float = 1000000):
        self.symbols = symbols
        self.initial_capital = initial_capital
        self.positions = {symbol: 0 for symbol in symbols}
        self.cash = initial_capital
        self.equity_curve = []
        
    def update_portfolio_value(self, prices: Dict[str, float]) -> float:
        """Calculate current portfolio value"""
        position_values = {symbol: self.positions[symbol] * prices.get(symbol, 0) 
                          for symbol in self.symbols}
        total_position_value = sum(position_values.values())
        total_value = self.cash + total_position_value
        
        return total_value
    
    def rebalance_portfolio(self, target_weights: Dict[str, float], prices: Dict[str, float]):
        """Rebalance portfolio to target weights"""
        current_value = self.update_portfolio_value(prices)
        
        for symbol in self.symbols:
            target_value = current_value * target_weights.get(symbol, 0)
            current_position_value = self.positions[symbol] * prices.get(symbol, 0)
            
            # Calculate required trade
            trade_value = target_value - current_position_value
            if prices.get(symbol, 0) > 0:
                shares_to_trade = int(trade_value / prices[symbol])
                
                # Update positions and cash
                self.positions[symbol] += shares_to_trade
                self.cash -= shares_to_trade * prices[symbol]
    
    def add_equity_point(self, date: datetime, prices: Dict[str, float]):
        """Add point to equity curve"""
        portfolio_value = self.update_portfolio_value(prices)
        self.equity_curve.append({
            'date': date,
            'portfolio_value': portfolio_value,
            'cash': self.cash,
            'positions': self.positions.copy()
        })
    
    def get_performance_stats(self) -> Dict[str, float]:
        """Calculate portfolio performance statistics"""
        if len(self.equity_curve) < 2:
            return {}
            
        values = [point['portfolio_value'] for point in self.equity_curve]
        returns = pd.Series(values).pct_change().dropna()
        
        total_return = (values[-1] / values[0]) - 1
        volatility = returns.std() * np.sqrt(252)
        sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
        
        # Max drawdown
        cumulative = pd.Series(values)
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min()
        
        return {
            'total_return': total_return,
            'annualized_volatility': volatility,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'final_value': values[-1]
        }

# Test portfolio management
print("💼 Testing Portfolio Management:")

# Initialize portfolio
symbols = ['SPY', 'QQQ', 'GLD', 'TLT']
portfolio = PortfolioManager(symbols, initial_capital=1000000)

# Simulate price evolution and rebalancing
np.random.seed(42)
n_days = 252
dates = pd.date_range('2024-01-01', periods=n_days, freq='D')

# Initial prices
initial_prices = {'SPY': 450, 'QQQ': 380, 'GLD': 180, 'TLT': 95}
current_prices = initial_prices.copy()

# Target weights (equal weight)
target_weights = {symbol: 0.25 for symbol in symbols}

# Simulate daily price changes and monthly rebalancing
for i, date in enumerate(dates):
    # Update prices with random walk
    for symbol in symbols:
        price_change = np.random.normal(0.0005, 0.02)  # Daily return
        current_prices[symbol] *= (1 + price_change)
    
    # Monthly rebalancing (every 21 trading days)
    if i % 21 == 0:
        portfolio.rebalance_portfolio(target_weights, current_prices)
    
    # Record portfolio state
    portfolio.add_equity_point(date, current_prices)

# Get performance statistics
performance = portfolio.get_performance_stats()

print(f"Portfolio Performance (1 Year Simulation):")
print(f"  Initial Capital: ${portfolio.initial_capital:,.0f}")
print(f"  Final Value: ${performance['final_value']:,.0f}")
print(f"  Total Return: {performance['total_return']*100:.2f}%")
print(f"  Annualized Volatility: {performance['annualized_volatility']*100:.2f}%")
print(f"  Sharpe Ratio: {performance['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {performance['max_drawdown']*100:.2f}%")

print(f"\nFinal Positions:")
for symbol, shares in portfolio.positions.items():
    value = shares * current_prices[symbol]
    print(f"  {symbol}: {shares} shares (${value:,.0f})")
print(f"  Cash: ${portfolio.cash:,.0f}")

print("\n✅ Portfolio management module working")


💼 Testing Portfolio Management:
Portfolio Performance (1 Year Simulation):
  Initial Capital: $1,000,000
  Final Value: $1,233,871
  Total Return: 23.39%
  Annualized Volatility: 15.92%
  Sharpe Ratio: 1.405
  Max Drawdown: -11.29%

Final Positions:
  SPY: 685 shares ($329,186)
  QQQ: 663 shares ($299,697)
  GLD: 1329 shares ($289,098)
  TLT: 2326 shares ($315,660)
  Cash: $230

✅ Portfolio management module working


## Summary: Python Fundamentals for Turtle Trading

### 🎯 **Core Components Built**

#### **1. Financial Metrics (`FinancialMetrics`)**
- **Returns Calculation**: Simple and log returns
- **Risk Metrics**: Volatility, Sharpe ratio, Calmar ratio
- **Drawdown Analysis**: Maximum drawdown with dates
- **Performance Statistics**: Comprehensive performance measurement

#### **2. Technical Indicators (`TechnicalIndicators`)**
- **Donchian Channels**: Core Turtle Trading breakout system
- **ATR (Average True Range)**: Volatility-based position sizing
- **Moving Averages**: Trend analysis and filtering
- **Breakout Signals**: Complete entry/exit signal generation

#### **3. Risk Management (`RiskManagement`)**
- **ATR-based Position Sizing**: True Turtle methodology
- **Portfolio Allocation**: Equal weight and custom allocations
- **Position Limits**: Risk controls and sector limits
- **Correlation Analysis**: Diversification measurement

#### **4. Portfolio Management (`PortfolioManager`)**
- **Multi-asset Portfolio**: Track positions across instruments
- **Rebalancing Logic**: Automated portfolio rebalancing
- **Equity Curve**: Performance tracking over time
- **Performance Analytics**: Complete performance statistics

### 🚀 **Ready for Strategy Implementation**

These fundamental building blocks provide everything needed to implement and backtest the Turtle Trading strategy:

- ✅ **Data Processing**: Efficient handling of multi-asset time series
- ✅ **Signal Generation**: Donchian breakout entry/exit signals  
- ✅ **Risk Management**: ATR-based position sizing with portfolio limits
- ✅ **Performance Measurement**: Comprehensive analytics and reporting
- ✅ **Portfolio Management**: Multi-asset position tracking and rebalancing

### 🔧 **Bug Fix Applied**
- **Fixed numpy array issue**: Converted price arrays to pandas Series before using `.shift()` method
- **Improved data generation**: More robust OHLC data creation with proper pandas operations

The next stages will combine these components into a complete backtesting framework for evaluating Turtle Trading performance in modern markets.


# Stage 03: Python Fundamentals for Quantitative Finance
**Project:** Turtle Trading Strategy Research  
**Author:** Panwei Hu  
**Date:** 2025-08-17

## Objectives
- Demonstrate core Python skills for quantitative finance
- Implement financial calculations and time series operations
- Build foundation functions for Turtle Trading strategy
- Practice data manipulation and analysis patterns
- Create reusable components for the research pipeline

## Focus Areas
- **Time Series Analysis**: Price data, returns, volatility calculations
- **Financial Metrics**: Sharpe ratio, drawdown, performance statistics
- **Technical Indicators**: Moving averages, Donchian channels, ATR
- **Risk Management**: Position sizing, portfolio allocation
- **Data Structures**: Efficient handling of multi-asset datasets


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up project environment
PROJECT_ROOT = Path('..').resolve()
import sys
sys.path.append(str(PROJECT_ROOT / 'src'))

print("🐢 Python Fundamentals for Turtle Trading")
print("="*50)

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("✅ Libraries imported and environment configured")


🐢 Python Fundamentals for Turtle Trading
✅ Libraries imported and environment configured


In [7]:
# Core Financial Calculations
class FinancialMetrics:
    """Core financial calculations for quantitative analysis"""
    
    @staticmethod
    def returns(prices: pd.Series, method: str = 'simple') -> pd.Series:
        """Calculate returns from price series"""
        if method == 'simple':
            return prices.pct_change()
        elif method == 'log':
            return np.log(prices / prices.shift(1))
        else:
            raise ValueError("Method must be 'simple' or 'log'")
    
    @staticmethod
    def volatility(returns: pd.Series, annualize: bool = True, periods: int = 252) -> float:
        """Calculate annualized volatility"""
        vol = returns.std()
        return vol * np.sqrt(periods) if annualize else vol
    
    @staticmethod
    def sharpe_ratio(returns: pd.Series, risk_free: float = 0.0, periods: int = 252) -> float:
        """Calculate Sharpe ratio"""
        excess_returns = returns - risk_free / periods
        return np.sqrt(periods) * excess_returns.mean() / returns.std()
    
    @staticmethod
    def max_drawdown(prices: pd.Series) -> Dict[str, float]:
        """Calculate maximum drawdown and related metrics"""
        cumulative = (1 + FinancialMetrics.returns(prices)).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        
        max_dd = drawdown.min()
        max_dd_date = drawdown.idxmin()
        
        return {
            'max_drawdown': max_dd,
            'max_drawdown_date': max_dd_date,
            'current_drawdown': drawdown.iloc[-1]
        }
    
    @staticmethod
    def calmar_ratio(returns: pd.Series, periods: int = 252) -> float:
        """Calculate Calmar ratio (annual return / max drawdown)"""
        annual_return = (1 + returns).prod() ** (periods / len(returns)) - 1
        prices = (1 + returns).cumprod()
        max_dd = abs(FinancialMetrics.max_drawdown(prices)['max_drawdown'])
        
        return annual_return / max_dd if max_dd != 0 else np.inf

# Test with sample data
print("📊 Testing Financial Metrics:")

# Generate sample price data
np.random.seed(42)
dates = pd.date_range('2020-01-01', '2024-12-31', freq='D')
returns = np.random.normal(0.0005, 0.02, len(dates))  # Daily returns with drift
prices = pd.Series(100 * (1 + returns).cumprod(), index=dates, name='price')

# Calculate metrics
sample_returns = FinancialMetrics.returns(prices)
vol = FinancialMetrics.volatility(sample_returns)
sharpe = FinancialMetrics.sharpe_ratio(sample_returns)
dd_info = FinancialMetrics.max_drawdown(prices)
calmar = FinancialMetrics.calmar_ratio(sample_returns)

print(f"Sample period: {prices.index[0].date()} to {prices.index[-1].date()}")
print(f"Total return: {(prices.iloc[-1] / prices.iloc[0] - 1) * 100:.2f}%")
print(f"Annualized volatility: {vol * 100:.2f}%")
print(f"Sharpe ratio: {sharpe:.3f}")
print(f"Maximum drawdown: {dd_info['max_drawdown'] * 100:.2f}%")
print(f"Calmar ratio: {calmar:.3f}")

print("\n✅ Financial metrics module working")


📊 Testing Financial Metrics:
Sample period: 2020-01-01 to 2024-12-31
Total return: 724.97%
Annualized volatility: 31.43%
Sharpe ratio: 1.084
Maximum drawdown: -42.33%
Calmar ratio: 0.798

✅ Financial metrics module working


In [8]:
# Technical Indicators for Turtle Trading
class TechnicalIndicators:
    """Technical indicators specifically for Turtle Trading strategy"""
    
    @staticmethod
    def donchian_channels(prices: pd.Series, period: int = 20) -> Dict[str, pd.Series]:
        """Calculate Donchian channel (key component of Turtle system)"""
        high_channel = prices.rolling(window=period).max()
        low_channel = prices.rolling(window=period).min()
        mid_channel = (high_channel + low_channel) / 2
        
        return {
            'upper': high_channel,
            'lower': low_channel,
            'middle': mid_channel
        }
    
    @staticmethod
    def atr(high: pd.Series, low: pd.Series, close: pd.Series, period: int = 20) -> pd.Series:
        """Calculate Average True Range (ATR) for position sizing"""
        # True Range calculation
        tr1 = high - low
        tr2 = abs(high - close.shift(1))
        tr3 = abs(low - close.shift(1))
        
        true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = true_range.rolling(window=period).mean()
        
        return atr
    
    @staticmethod
    def moving_average(prices: pd.Series, period: int, method: str = 'simple') -> pd.Series:
        """Calculate moving averages"""
        if method == 'simple':
            return prices.rolling(window=period).mean()
        elif method == 'exponential':
            return prices.ewm(span=period).mean()
        else:
            raise ValueError("Method must be 'simple' or 'exponential'")
    
    @staticmethod
    def breakout_signals(prices: pd.Series, entry_period: int = 20, exit_period: int = 10) -> Dict[str, pd.Series]:
        """Generate Turtle Trading breakout signals"""
        # Entry signals
        entry_channels = TechnicalIndicators.donchian_channels(prices, entry_period)
        long_entry = prices > entry_channels['upper'].shift(1)
        short_entry = prices < entry_channels['lower'].shift(1)
        
        # Exit signals  
        exit_channels = TechnicalIndicators.donchian_channels(prices, exit_period)
        long_exit = prices < exit_channels['lower'].shift(1)
        short_exit = prices > exit_channels['upper'].shift(1)
        
        return {
            'long_entry': long_entry,
            'short_entry': short_entry,
            'long_exit': long_exit,
            'short_exit': short_exit,
            'entry_upper': entry_channels['upper'],
            'entry_lower': entry_channels['lower'],
            'exit_upper': exit_channels['upper'],
            'exit_lower': exit_channels['lower']
        }

# Test technical indicators
print("🔧 Testing Technical Indicators:")

# Create more realistic price data with OHLC
np.random.seed(42)
n_days = 500
base_price = 100
price_changes = np.random.normal(0, 0.02, n_days)
closes = base_price * (1 + price_changes).cumprod()

# Generate OHLC data
highs = closes * (1 + np.abs(np.random.normal(0, 0.01, n_days)))
lows = closes * (1 - np.abs(np.random.normal(0, 0.01, n_days)))
opens = closes.shift(1).fillna(base_price)

dates = pd.date_range('2023-01-01', periods=n_days, freq='D')
ohlc_data = pd.DataFrame({
    'open': opens,
    'high': highs, 
    'low': lows,
    'close': closes
}, index=dates)

# Test Donchian channels
donchian = TechnicalIndicators.donchian_channels(ohlc_data['close'], 20)
print(f"Donchian channels calculated for {len(donchian['upper'])} periods")

# Test ATR
atr_values = TechnicalIndicators.atr(ohlc_data['high'], ohlc_data['low'], ohlc_data['close'])
avg_atr = atr_values.mean()
print(f"Average ATR: {avg_atr:.4f} ({avg_atr/ohlc_data['close'].mean()*100:.2f}% of price)")

# Test breakout signals
signals = TechnicalIndicators.breakout_signals(ohlc_data['close'])
long_signals = signals['long_entry'].sum()
short_signals = signals['short_entry'].sum()
print(f"Generated {long_signals} long entry signals and {short_signals} short entry signals")

# Test moving averages
ma_20 = TechnicalIndicators.moving_average(ohlc_data['close'], 20)
ma_50 = TechnicalIndicators.moving_average(ohlc_data['close'], 50)
print(f"Moving averages: MA20={ma_20.iloc[-1]:.2f}, MA50={ma_50.iloc[-1]:.2f}")

print("\n✅ Technical indicators module working")


🔧 Testing Technical Indicators:


AttributeError: 'numpy.ndarray' object has no attribute 'shift'

In [ ]:
# Risk Management and Position Sizing
class RiskManagement:
    """Risk management utilities for Turtle Trading"""
    
    @staticmethod
    def position_size_atr(account_value: float, risk_per_trade: float, atr: float, price: float) -> int:
        """Calculate position size based on ATR (Turtle method)"""
        # Dollar risk per trade
        dollar_risk = account_value * risk_per_trade
        
        # Position size = Dollar risk / (ATR * multiplier)
        # Using 2 * ATR as stop distance (common Turtle approach)
        stop_distance = 2 * atr
        position_size = int(dollar_risk / stop_distance)
        
        return max(position_size, 0)
    
    @staticmethod
    def portfolio_allocation(symbols: List[str], equal_weight: bool = True, 
                           custom_weights: Optional[Dict[str, float]] = None) -> Dict[str, float]:
        """Calculate portfolio allocation weights"""
        if equal_weight:
            weight = 1.0 / len(symbols)
            return {symbol: weight for symbol in symbols}
        elif custom_weights:
            total_weight = sum(custom_weights.values())
            return {symbol: weight/total_weight for symbol, weight in custom_weights.items()}
        else:
            raise ValueError("Must specify either equal_weight=True or custom_weights")
    
    @staticmethod
    def max_position_limits(account_value: float, max_risk_per_position: float = 0.02,
                          max_sector_risk: float = 0.20) -> Dict[str, float]:
        """Calculate maximum position and sector limits"""
        return {
            'max_position_value': account_value * max_risk_per_position,
            'max_sector_value': account_value * max_sector_risk,
            'max_portfolio_risk': account_value * 0.02  # 2% portfolio stop
        }
    
    @staticmethod
    def correlation_analysis(returns_df: pd.DataFrame) -> Dict[str, Union[pd.DataFrame, float]]:
        """Analyze correlations between assets for diversification"""
        corr_matrix = returns_df.corr()
        
        # Average correlation (excluding diagonal)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        avg_correlation = corr_matrix.where(mask).stack().mean()
        
        # Find highly correlated pairs (>0.7)
        high_corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > 0.7:
                    high_corr_pairs.append({
                        'asset1': corr_matrix.columns[i],
                        'asset2': corr_matrix.columns[j], 
                        'correlation': corr_val
                    })
        
        return {
            'correlation_matrix': corr_matrix,
            'average_correlation': avg_correlation,
            'high_correlation_pairs': high_corr_pairs
        }

# Test risk management functions
print("⚖️  Testing Risk Management:")

# Portfolio setup
account_value = 1000000  # $1M account
symbols = ['SPY', 'QQQ', 'GLD', 'TLT', 'EEM']
risk_per_trade = 0.01  # 1% risk per trade

# Test position sizing
test_atr = 2.5
test_price = 150.0
position_size = RiskManagement.position_size_atr(account_value, risk_per_trade, test_atr, test_price)
print(f"Position size for ATR={test_atr}, Price=${test_price}: {position_size} shares")
print(f"Position value: ${position_size * test_price:,.0f} ({position_size * test_price / account_value * 100:.1f}% of account)")

# Test portfolio allocation
equal_weights = RiskManagement.portfolio_allocation(symbols, equal_weight=True)
print(f"\nEqual weight allocation: {equal_weights}")

custom_weights = {'SPY': 0.3, 'QQQ': 0.2, 'GLD': 0.2, 'TLT': 0.2, 'EEM': 0.1}
custom_allocation = RiskManagement.portfolio_allocation(symbols, equal_weight=False, custom_weights=custom_weights)
print(f"Custom allocation: {custom_allocation}")

# Test position limits
limits = RiskManagement.max_position_limits(account_value)
print(f"\nPosition limits:")
for limit_type, value in limits.items():
    print(f"  {limit_type}: ${value:,.0f}")

# Test correlation analysis with synthetic data
np.random.seed(42)
n_days = 252
returns_data = {}

# Create correlated returns for testing
base_returns = np.random.normal(0, 0.02, n_days)
for i, symbol in enumerate(symbols):
    # Add some correlation structure
    correlation_factor = 0.3 + i * 0.1
    symbol_returns = correlation_factor * base_returns + (1 - correlation_factor) * np.random.normal(0, 0.02, n_days)
    returns_data[symbol] = symbol_returns

returns_df = pd.DataFrame(returns_data)
corr_analysis = RiskManagement.correlation_analysis(returns_df)

print(f"\nCorrelation Analysis:")
print(f"Average correlation: {corr_analysis['average_correlation']:.3f}")
print(f"High correlation pairs: {len(corr_analysis['high_correlation_pairs'])}")

for pair in corr_analysis['high_correlation_pairs']:
    print(f"  {pair['asset1']} - {pair['asset2']}: {pair['correlation']:.3f}")

print("\n✅ Risk management module working")


In [ ]:
# Data Processing and Portfolio Management
class PortfolioManager:
    """Portfolio management utilities for multi-asset strategies"""
    
    def __init__(self, symbols: List[str], initial_capital: float = 1000000):
        self.symbols = symbols
        self.initial_capital = initial_capital
        self.positions = {symbol: 0 for symbol in symbols}
        self.cash = initial_capital
        self.equity_curve = []
        
    def update_portfolio_value(self, prices: Dict[str, float]) -> float:
        """Calculate current portfolio value"""
        position_values = {symbol: self.positions[symbol] * prices.get(symbol, 0) 
                          for symbol in self.symbols}
        total_position_value = sum(position_values.values())
        total_value = self.cash + total_position_value
        
        return total_value
    
    def rebalance_portfolio(self, target_weights: Dict[str, float], prices: Dict[str, float]):
        """Rebalance portfolio to target weights"""
        current_value = self.update_portfolio_value(prices)
        
        for symbol in self.symbols:
            target_value = current_value * target_weights.get(symbol, 0)
            current_position_value = self.positions[symbol] * prices.get(symbol, 0)
            
            # Calculate required trade
            trade_value = target_value - current_position_value
            if prices.get(symbol, 0) > 0:
                shares_to_trade = int(trade_value / prices[symbol])
                
                # Update positions and cash
                self.positions[symbol] += shares_to_trade
                self.cash -= shares_to_trade * prices[symbol]
    
    def add_equity_point(self, date: datetime, prices: Dict[str, float]):
        """Add point to equity curve"""
        portfolio_value = self.update_portfolio_value(prices)
        self.equity_curve.append({
            'date': date,
            'portfolio_value': portfolio_value,
            'cash': self.cash,
            'positions': self.positions.copy()
        })
    
    def get_performance_stats(self) -> Dict[str, float]:
        """Calculate portfolio performance statistics"""
        if len(self.equity_curve) < 2:
            return {}
            
        values = [point['portfolio_value'] for point in self.equity_curve]
        returns = pd.Series(values).pct_change().dropna()
        
        total_return = (values[-1] / values[0]) - 1
        volatility = returns.std() * np.sqrt(252)
        sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
        
        # Max drawdown
        cumulative = pd.Series(values)
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min()
        
        return {
            'total_return': total_return,
            'annualized_volatility': volatility,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'final_value': values[-1]
        }

# Test portfolio management
print("💼 Testing Portfolio Management:")

# Initialize portfolio
symbols = ['SPY', 'QQQ', 'GLD', 'TLT']
portfolio = PortfolioManager(symbols, initial_capital=1000000)

# Simulate price evolution and rebalancing
np.random.seed(42)
n_days = 252
dates = pd.date_range('2024-01-01', periods=n_days, freq='D')

# Initial prices
initial_prices = {'SPY': 450, 'QQQ': 380, 'GLD': 180, 'TLT': 95}
current_prices = initial_prices.copy()

# Target weights (equal weight)
target_weights = {symbol: 0.25 for symbol in symbols}

# Simulate daily price changes and monthly rebalancing
for i, date in enumerate(dates):
    # Update prices with random walk
    for symbol in symbols:
        price_change = np.random.normal(0.0005, 0.02)  # Daily return
        current_prices[symbol] *= (1 + price_change)
    
    # Monthly rebalancing (every 21 trading days)
    if i % 21 == 0:
        portfolio.rebalance_portfolio(target_weights, current_prices)
    
    # Record portfolio state
    portfolio.add_equity_point(date, current_prices)

# Get performance statistics
performance = portfolio.get_performance_stats()

print(f"Portfolio Performance (1 Year Simulation):")
print(f"  Initial Capital: ${portfolio.initial_capital:,.0f}")
print(f"  Final Value: ${performance['final_value']:,.0f}")
print(f"  Total Return: {performance['total_return']*100:.2f}%")
print(f"  Annualized Volatility: {performance['annualized_volatility']*100:.2f}%")
print(f"  Sharpe Ratio: {performance['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {performance['max_drawdown']*100:.2f}%")

print(f"\nFinal Positions:")
for symbol, shares in portfolio.positions.items():
    value = shares * current_prices[symbol]
    print(f"  {symbol}: {shares} shares (${value:,.0f})")
print(f"  Cash: ${portfolio.cash:,.0f}")

print("\n✅ Portfolio management module working")


## Summary: Python Fundamentals for Turtle Trading

### 🎯 **Core Components Built**

#### **1. Financial Metrics (`FinancialMetrics`)**
- **Returns Calculation**: Simple and log returns
- **Risk Metrics**: Volatility, Sharpe ratio, Calmar ratio
- **Drawdown Analysis**: Maximum drawdown with dates
- **Performance Statistics**: Comprehensive performance measurement

#### **2. Technical Indicators (`TechnicalIndicators`)**
- **Donchian Channels**: Core Turtle Trading breakout system
- **ATR (Average True Range)**: Volatility-based position sizing
- **Moving Averages**: Trend analysis and filtering
- **Breakout Signals**: Complete entry/exit signal generation

#### **3. Risk Management (`RiskManagement`)**
- **ATR-based Position Sizing**: True Turtle methodology
- **Portfolio Allocation**: Equal weight and custom allocations
- **Position Limits**: Risk controls and sector limits
- **Correlation Analysis**: Diversification measurement

#### **4. Portfolio Management (`PortfolioManager`)**
- **Multi-asset Portfolio**: Track positions across instruments
- **Rebalancing Logic**: Automated portfolio rebalancing
- **Equity Curve**: Performance tracking over time
- **Performance Analytics**: Complete performance statistics

### 🚀 **Ready for Strategy Implementation**

These fundamental building blocks provide everything needed to implement and backtest the Turtle Trading strategy:

- ✅ **Data Processing**: Efficient handling of multi-asset time series
- ✅ **Signal Generation**: Donchian breakout entry/exit signals  
- ✅ **Risk Management**: ATR-based position sizing with portfolio limits
- ✅ **Performance Measurement**: Comprehensive analytics and reporting
- ✅ **Portfolio Management**: Multi-asset position tracking and rebalancing

The next stages will combine these components into a complete backtesting framework for evaluating Turtle Trading performance in modern markets.
